In [1]:
import java.nio.file.Paths
import ammonite.ops._
import ammonite._

val p = Paths.get("/home/ixaxaar/.m2/repository/ml/dmlc/mxnet/mxnet-full_2.11-linux-x86_64-gpu/0.1.2-SNAPSHOT/mxnet-full_2.11-linux-x86_64-gpu-0.1.2-SNAPSHOT.jar")

interp.load.cp(FilePath(p).asInstanceOf[ammonite.ops.Path])

import java.nio.file.Paths

import ammonite.ops._

import ammonite._


p: java.nio.file.Path = /home/ixaxaar/.m2/repository/ml/dmlc/mxnet/mxnet-full_2.11-linux-x86_64-gpu/0.1.2-SNAPSHOT/mxnet-full_2.11-linux-x86_64-gpu-0.1.2-SNAPSHOT.jar

In [20]:
interp.load.ivy("org.typelevel" %% "cats" % "0.9.0")
import cats._, cats.syntax._, cats.implicits._, cats.instances.all._

import cats._, cats.syntax._, cats.implicits._, cats.instances.all._

In [2]:
import ml.dmlc.mxnet._
import ml.dmlc.mxnet.{ Symbol => T }
import ml.dmlc.mxnet.{ NDArray => np }

import ml.dmlc.mxnet._

import ml.dmlc.mxnet.{ Symbol => T }

import ml.dmlc.mxnet.{ NDArray => np }

In [4]:
T.FullyConnected _

res3: (String, Map[String, String]) => Seq[Symbol] => Map[String, Any] => Symbol = <function2>

In [5]:
T.Variable _

res4: (String, Map[String, String]) => Symbol = <function2>

In [16]:
val arr = NDArray.array(Array(1f, 2f, 1f, 100000f), shape = Shape(2, 2))
NDArray.argmax_channel(arr).toArray

arr: NDArray = ml.dmlc.mxnet.NDArray@104b9c6d
res15_1: Array[Float] = Array(1.0F, 1.0F)

In [7]:
NDArray.argmax_channel _

res6: Map[String, Any] => Seq[Any] => NDArrayFuncReturn = <function1>

In [17]:
val sentenceSize = 10
val numEmbed = 5
val batchSize = 3
val numLabel = 2
val filterList = Array(3, 4, 5)
val numFilter = 100
val dropout = 0.5f

sentenceSize: Int = 10
numEmbed: Int = 5
batchSize: Int = 3
numLabel: Int = 2
filterList: Array[Int] = Array(3, 4, 5)
numFilter: Int = 100
dropout: Float = 0.5F

In [22]:
val inputX = T.Variable("data")
val inputY = T.Variable("softmax_label")

// convolution -> relu -> max pooling for all the filter sizes
val polledOutputs: List[T] = filterList.map { filterSize =>
  T.Convolution()()(Map(
    "data" -> inputX,
    "kernel" -> s"($filterSize, $numEmbed)",
    "num_filter" -> numFilter
  )).some.map{ conv =>

    T.Activation()()(Map("data" -> conv, "act_type" -> "relu"))
  }.map{ relu =>

    T.Pooling()()(Map(
      "data" -> relu,
      "pool_type" -> "max",
      "kernel" -> s"(${sentenceSize - filterSize + 1}, 1)",
      "stride" -> "(1,1)"
    ))
  }
}.toList.flatten match {
  case Nil => List[T]()
  case x   => x
}

val totalFilters = numFilter * filterList.length

// concat(filters) -> dropout -> dense -> softmax
val mod = T.Concat()(polledOutputs: _*)(Map("dim" -> 1)).some.map{ concat =>
  T.Reshape()()(Map(
    "data" -> concat,
    "target_shape" -> s"($batchSize, $totalFilters)"
  ))
}.map{ hPool =>

  if (dropout > 0f) T.Dropout()()(Map("data" -> hPool, "p" -> dropout))
  else hPool
}.map{ hDrop =>

  T.FullyConnected()()(Map("data" -> hDrop, "num_hidden" -> numLabel))
}.map{ fc =>

  T.SoftmaxOutput()()(Map("data" -> fc, "label" -> inputY))
}

inputX: Symbol = ml.dmlc.mxnet.Symbol@65807aa4
inputY: Symbol = ml.dmlc.mxnet.Symbol@4899ca3a
polledOutputs: List[Symbol] = List(
  ml.dmlc.mxnet.Symbol@24f2de8b,
  ml.dmlc.mxnet.Symbol@777206ae,
  ml.dmlc.mxnet.Symbol@11c117be
)
totalFilters: Int = 300
mod: Option[Symbol] = Some(ml.dmlc.mxnet.Symbol@170fc7a6)

In [23]:
case class CNNModel(
  cnnExec:  Executor,
  symbol:   Symbol,
  data:     np,
  label:    np,
  argsDict: Map[String, np],
  gradDict: Map[String, np]
)

defined class CNNModel

In [28]:
val ctx = Context.gpu(0)
val model = mod.map{ cnn =>
  val argNames: List[String] = cnn.listArguments.toList
  println(argNames)
  val auxNames: List[String] = cnn.listAuxiliaryStates.toList
  println(auxNames)

  val (argShapes, outShapes, auxShapes) = cnn.inferShape(
    Map("data" -> Shape(batchSize, 1, sentenceSize, numEmbed))
  )
    println((argShapes, outShapes, auxShapes))

  val argsDict: Map[String, np] = argNames.zip(argShapes.map(np.zeros(_, ctx))).toMap

  val argsGradDict: Map[String, np] = argNames.zip(argShapes)
    .filter(x => x._1 != "softmax_label" && x._1 != "data")
    .map(x => x._1 -> np.zeros(x._2, ctx)).toMap

  val auxDict: Map[String, np] = auxNames.zip(auxShapes.map(np.zeros(_, ctx))).toMap
  val cnnExec: Executor = cnn.bind(ctx, argsDict, argsGradDict, "add", auxDict, null, null)

  val data = argsDict("data")
  val label = argsDict("softmax_label")
  CNNModel(cnnExec, cnn, data, label, argsDict, argsGradDict)
}

List(data, convolution3_weight, convolution3_bias, convolution4_weight, convolution4_bias, convolution5_weight, convolution5_bias, fullyconnected1_weight, fullyconnected1_bias, softmax_label)
List()
(Vector((3,1,10,5), (100,1,3,5), (100), (100,1,4,5), (100), (100,1,5,5), (100), (2,300), (2), (3)),Vector((3,2)),Vector())


ctx: Context = gpu(0)
model: Option[CNNModel] = Some(
  CNNModel(
    ml.dmlc.mxnet.Executor@5d37a7be,
    ml.dmlc.mxnet.Symbol@170fc7a6,
    ml.dmlc.mxnet.NDArray@ad707aa8,
    ml.dmlc.mxnet.NDArray@fd93d551,
    Map(
      "convolution3_weight" -> ml.dmlc.mxnet.NDArray@13f19148,
      "softmax_label" -> ml.dmlc.mxnet.NDArray@28966459,
      "convolution5_weight" -> ml.dmlc.mxnet.NDArray@471696ad,
      "convolution4_bias" -> ml.dmlc.mxnet.NDArray@f09c6d93,
      "fullyconnected1_weight" -> ml.dmlc.mxnet.NDArray@b2a54285,
...

In [29]:
model.map{ m => m.symbol }

res28: Option[Symbol] = Some(ml.dmlc.mxnet.Symbol@170fc7a6)

# Data

In [67]:
np.ones(2, 3)
np.zeros(3, 3)
val x = np.array(List(1,2,3,4).toArray.map{ _.toFloat }, shape=Shape(2,2))
x.shape

res66_0: NDArray = ml.dmlc.mxnet.NDArray@b56d6c4f
res66_1: NDArray = ml.dmlc.mxnet.NDArray@7524520f
x: NDArray = ml.dmlc.mxnet.NDArray@e4218aff
res66_3: Shape = (2,2)

In [28]:
val x = np.ones(100, 100) + np.zeros(100, 100)
val y = np.ones(100, 100) * np.zeros(100, 100)
val z = np.ones(100, 100) / 3 + 1
x += (y + z)
np.dot(x, z).toArray

x: NDArray = ml.dmlc.mxnet.NDArray@907ad41
y: NDArray = ml.dmlc.mxnet.NDArray@713fff9
z: NDArray = ml.dmlc.mxnet.NDArray@e7bd815c
res27_3: NDArray = ml.dmlc.mxnet.NDArray@10b08418
res27_4: Array[Float] = Array(
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
  311.11115F,
...

In [54]:
val x = np.ones(10, 5)
x.shape
x.toArray.length
x.slice(0).shape
x.slice(1).toArray.length
x.slice(0, 2).toArray.length
x.reshape(Shape(5, 10)).shape

x: NDArray = ml.dmlc.mxnet.NDArray@fd2cfa66
res53_1: Shape = (10,5)
res53_2: Int = 50
res53_3: Shape = (1,5)
res53_4: Int = 5
res53_5: Int = 10
res53_6: Shape = (5,10)

In [58]:
np.save("/home/ixaxaar/src/notebooks/ndarray.mxnet", List(x))
val y = np.load("/home/ixaxaar/src/notebooks/ndarray.mxnet")._2.head.shape

y: Shape = (10,5)

In [66]:
x.context
val ctx = Context.gpu(0)
np.array(x.toArray, x.shape, ctx).context

val gpu_x = np.zeros(x.shape, ctx)
x.copyTo(gpu_x)
gpu_x.context

res65_0: Context = cpu(0)
ctx: Context = gpu(0)
res65_2: Context = gpu(0)
gpu_x: NDArray = ml.dmlc.mxnet.NDArray@2a9c2d46
res65_4: NDArray = ml.dmlc.mxnet.NDArray@2ea112fa
res65_5: Context = gpu(0)

# Model API

In [10]:
val data = T.Variable("data")
val f1 = T.FullyConnected(name="f1")()(Map("data" -> data, "num_hidden" -> 128))
val a1 = T.Activation(name="relu1")()(Map("data" -> f1, "act_type" -> "relu"))
val f2 = T.FullyConnected(name="f2")()(Map("data" -> a1, "num_hidden" -> 128))
val softmax = T.SoftmaxOutput(name="softmax")()(Map("data" -> f2))

data: Symbol = ml.dmlc.mxnet.Symbol@687797cb
f1: Symbol = ml.dmlc.mxnet.Symbol@2841c40d
a1: Symbol = ml.dmlc.mxnet.Symbol@c690d46
f2: Symbol = ml.dmlc.mxnet.Symbol@76dcd821
softmax: Symbol = ml.dmlc.mxnet.Symbol@2241c0fe

In [10]:
val model = FeedForward.newBuilder(softmax)
    .setContext(Context.gpu())
    .setNumEpoch(10)
    .setOptimizer(new Adam())
    .setTrainData(trainDataIter)
    .setEvalData(evalDataIter)
    .build()

cmd10.sc:4: not found: type Adam
    .setOptimizer(new Adam())
                      ^cmd10.sc:5: not found: value trainDataIter
    .setTrainData(trainDataIter)
                  ^cmd10.sc:6: not found: value evalDataIter
    .setEvalData(evalDataIter)
                 ^

: 

In [10]:
model.fit(trainData = train)

cmd10.sc:1: not found: value model
val res10 = model.fit(trainData = train)
            ^cmd10.sc:1: not found: value trainData
val res10 = model.fit(trainData = train)
                      ^

: 

In [10]:
val modelPrefix: String = "checkpt"
val num_epoch = 100
Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)

// load model back
val model_loaded = FeedForward.load(modelPrefix, num_epoch)

cmd10.sc:3: not found: value epoch
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                ^cmd10.sc:3: not found: value symbol
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                           ^cmd10.sc:3: not found: value argParams
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                                   ^cmd10.sc:3: not found: value auxStates
val res10_2 = Model.saveCheckpoint(modelPrefix, epoch + 1, symbol, argParams, auxStates)
                                                                              ^

: 